In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge

from sklearn.metrics import mean_squared_error

import pickle

In [3]:
def read_df(filename: str):
    df = pd.read_parquet(filename)

    df["lpep_pickup_datetime"] = pd.to_datetime(df["lpep_pickup_datetime"])
    df["lpep_dropoff_datetime"] = pd.to_datetime(df["lpep_dropoff_datetime"])

    df["duration"] = df["lpep_dropoff_datetime"] - df["lpep_pickup_datetime"] # our target

    td = df["duration"].iloc[0]
    df["duration"] = df["duration"].apply(lambda td: td.total_seconds() / 60) # turn it into minutes
    
    df = df[(df.duration >= 1) & (df.duration <= 60)] # remove outliers / errors

    # features we are going to use
    categorical = ["PULocationID", "DOLocationID"]
    numerical = ["trip_distance"]

    df[categorical] = df[categorical].astype(str) # ints that will be used as categorial features
    
    return df

In [4]:
df_train = read_df("../data/green_tripdata_2021-01.parquet") # train the model on january
df_val = read_df("../data/green_tripdata_2021-02.parquet")   # validate the model on febuary

In [11]:
round(df_train['duration'].std(),2)

11.56

In [6]:
# creating a better feature on existing columns = better model (reduce mse)
df_train["PU_DO"] = df_train["PULocationID"] + '_' + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + '_' + df_val["DOLocationID"]

In [7]:
# transform dicts into feature matrices
dv = DictVectorizer() # performs auto one-hot encoding on categorial features

categorical = ["PU_DO", "PULocationID", "DOLocationID"]
numerical = ["trip_distance"]

train_dict = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dict) # convert dict to matrix

val_dict = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dict)


In [8]:
# define our target on train and validation sets
target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.397725977550327

In [10]:
# by saving both objects, dv can apply the transformations to future datasets 
# and lr can make the predictions after that

with open("../models/lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)

Notebook problems (main problems: reproducibility and automation)

- Hard to keep track of experiments    -> Experiment Tracking (MLFlow)
- Track model versions and its metrics -> Model Registry (MLFlow)
- Code execution order                 -> Pipelines (steps: load - vectorize - train...)
- Deployment (serve the pickle to users)
- Monitoring (drops in performance -> alerts and automatic retraining -> serve new model)    